In [1]:
import pandas as pd
import numpy as np
from scipy.spatial.distance import cdist

# 假设有两个DataFrame: df1 和 df2，分别包含站点名称和坐标信息
# df1 = pd.DataFrame({'name': [...], 'latitude': [...], 'longitude': [...]})
# df2 = pd.DataFrame({'name': [...], 'latitude': [...], 'longitude': [...]})

# 计算两个DataFrame中所有站点之间的距离矩阵
coords_1 = charge_value[['lon', 'lat']].values
coords_2 = data[['lon', 'lat']].values
distance_matrix = cdist(coords_1, coords_2, metric='euclidean')

# 找到每对距离最小的站点
charge_value['closest_index'] = np.argmin(distance_matrix, axis=1)
charge_value['min_distance'] = np.min(distance_matrix, axis=1)

# 可以设置一个阈值来过滤过远的站点
threshold = 0.01  # 设定距离阈值，比如 0.01 度
df1_filtered = charge_value[charge_value['min_distance'] <= threshold]

# 根据找到的最小距离索引将两个DataFrame合并
merged_df = pd.merge(df1_filtered, df2, left_on='closest_index', right_index=True, suffixes=('_df1', '_df2'))

# 删除临时列
merged_df.drop(columns=['closest_index', 'min_distance'], inplace=True)

# 输出合并后的结果
print(merged_df)


NameError: name 'df1' is not defined

In [11]:
import pandas as pd
import numpy as np
from scipy.spatial.distance import cdist
from scipy.optimize import linear_sum_assignment


data1 = {
    'name': ['A', 'B'],
    'lat': [41.51372219, 41.51422756],
    'lon': [122.6729427, 122.6695231]
}
data2 = {    'name': ['X', 'Y'],
    'lat': [41.5146335, 41.51405519],
    'lon': [122.6710758,122.6725509 ]
}

charge_value = pd.DataFrame(data1)
data = pd.DataFrame(data2)

coords_1 = charge_value[['lon', 'lat']].values
coords_2 = data[['lon', 'lat']].values
distance_matrix = cdist(coords_1, coords_2, metric='euclidean')

# 使用匈牙利算法进行最优匹配，确保每个 df2 的站点只匹配一次
row_ind, col_ind = linear_sum_assignment(distance_matrix)

# 创建一个新的 DataFrame 来存储匹配结果
matched_pairs = pd.DataFrame({
    'df1_index': row_ind,
    'df2_index': col_ind,
    'distance': distance_matrix[row_ind, col_ind]
})

# 设置一个距离阈值来过滤过远的站点
threshold = 0.01
matched_pairs_filtered = matched_pairs[matched_pairs['distance'] <= threshold]

# 根据找到的匹配索引将两个 DataFrame 合并
merged_df_optimal = pd.merge(charge_value.loc[matched_pairs_filtered['df1_index']], 
                             data.loc[matched_pairs_filtered['df2_index']].reset_index(), 
                             left_index=True, 
                             right_index=True, 
                             suffixes=('_df1', '_df2'))

# 删除临时列
merged_df_optimal.drop(columns=['index'], inplace=True)


merged_df_optimal

,name_df1,lat_df1,lon_df1,name_df2,lat_df2,lon_df2
0,A,41.513722,122.672943,Y,41.514055,122.672551
1,B,41.514228,122.669523,X,41.514634,122.671076
